## (20 pts) Use the estimated transition and emission parameters, implement the alternative max-marginal decoding algorithm. Clearly describe the steps of your algorithm in your report.
### Run the algorithm on the development sets EN/dev.in and FR/dev.in only. Write the outputs to EN/dev.p4.out and FR/dev.p4.out. Report the precision, recall and F scores for the outputs for both languages.
### Hint: the max-marginal decoding involves the implementation of the forward-backward algorithm.

In [1]:
import numpy as np 
import pandas as pd

## Emission and Transition functions

In [2]:
def get_data(filename):
    f = open(filename,'r')
    lines = f.readlines()
    datas = []
    
    start = 0
    for i in range(len(lines)):
        if lines[i] == '\n':
            datas.append(lines[start:i])
            start = i+1
        lines[i] = lines[i].replace('\n','')
        lines[i] = tuple(lines[i].split(' '))
        
    # check formatting
    for i in range(len(datas)):
        for j in range(len(datas[i])):
            #print datas[i][j]
            assert len(datas[i][j])==2
            
    
    for i in range(len(datas)):
        data = datas[i]
        x = [word[0] for word in data]
        y = [word[1] for word in data]
        datas[i] = [x,y]
    
    all_x = []
    for i in range(len(datas)):
        for j in range(len(datas[i][0])):
            all_x.append(datas[i][0][j])
    x_set = frozenset(all_x)
    
    all_y = []
    for i in range(len(datas)):
        for j in range(len(datas[i][0])):
            all_y.append(datas[i][1][j])
    y_set = frozenset(all_y)
    
    return dict(data=datas,x_set=x_set,y_set=y_set)

def get_unlabelled_data(filename):
    f = open(filename,'r')
    lines = f.readlines()
    datas = []
    
    start = 0
    for i in range(len(lines)):
        if lines[i] == '\n':
            datas.append(lines[start:i])
            start = i+1
        lines[i] = lines[i].replace('\n','')
            
    return datas

def get_transmission_params(data_dict):
    from_y = ['START'] + list(data_dict['y_set']) 
    to_y = list(data_dict['y_set']) + ['STOP']
    l = len(from_y)
    transmission_count = pd.DataFrame(np.zeros((l,l)),index=from_y,columns=to_y)

    datas = data_dict['data']
    for instance in datas:
        x_vector,y_vector = instance
        length = len(y_vector)
        for i in range(length+1):
            if i == 0 :
                transmission_count.loc['START',y_vector[0]] += 1

            elif i == length:
                transmission_count.loc[y_vector[i-1],'STOP'] +=1

            else:
                transmission_count.loc[y_vector[i-1],y_vector[i]] += 1

    y_count = transmission_count.sum(axis=1) 
    transmission_params = transmission_count
    for i in range(len(transmission_count.index)):
        transmission_params.iloc[i,:] /= transmission_params.iloc[i,:].sum()
    return transmission_params

def get_emission_counts(data_dict):
    """
    returns (DataFrame,Series) 
    an emission count (y->x) DataFrame and y count Series
    """
    data = data_dict['data']
    x_set = data_dict['x_set']
    y_set = data_dict['y_set']
    count_em_df = pd.DataFrame(np.zeros((len(x_set),len(y_set))),index=x_set,columns=y_set)
    count_y = pd.Series(np.zeros(len(y_set)),index=y_set)

    for instance in data:
        x_vector,y_vector = instance
        for i in range(len(x_vector)):
            x,y = x_vector[i],y_vector[i]
            count_em_df.loc[x,y]+=1
            count_y[y]+=1
    return count_em_df,count_y

def get_modified_counts(data_dict,k):
    count_em_df,count_y = get_emission_counts(data_dict)
    
    counts_x = count_em_df.sum(axis=1)
    fail = counts_x[counts_x<k]

    unk = count_em_df.loc[fail.index].sum(axis=0)
    unk.name = '#UNK#'
   
    modified_df = count_em_df.append(unk)
    modified_df = modified_df.drop(fail.index, axis=0) 
    
    return modified_df,count_y


def get_modified_emission_params(data_dict,k=3):
    """
    returns DataFrame representing conditional probabilities P(y|x)
    """
    count_em_df,count_y = get_modified_counts(data_dict,k)
    return count_em_df/count_y


## Max Marginal Functions

In [3]:
def get_forward_prob(x_vector,trans_params,em_params):
    
    states = trans_params.index.tolist()
    states.remove('START')
    states.remove('O')
    states = ['O']+states

    arr = np.zeros((len(states),len(x_vector))) *np.nan

    alpha = pd.DataFrame(arr,index=states,columns=x_vector)

    # base case
    for i in range(len(states)):
        alpha.iloc[i,0] = trans_params.loc['START',states[i]]
    
    # recursive case
    for i in range(1,len(x_vector)):
        for u in range(len(states)):
            summ = 0
            for v in range(len(states)):
                summ += alpha.iloc[v,i-1] * trans_params.loc[states[v],states[u]] * \
                        em_params.loc[x_vector[i],states[v]]
            alpha.loc[states[u],x_vector[i]] = summ
    
    return alpha

def get_backward_prob(x_vector,trans_params,em_params):

    states = trans_params.index.tolist()
    states.remove('START')
    states.remove('O')
    states = ['O']+states

    arr = np.zeros((len(states),len(x_vector))) *np.nan

    beta = pd.DataFrame(arr,index=states,columns=x_vector)

    # base case
    for i in range(len(states)):
        beta.iloc[i,len(x_vector)-1] = trans_params.loc[states[i],'STOP'] * \
                                       em_params.loc[x_vector[-1],states[i]]
     
    # recursive case
    for i in range(len(x_vector)-2,-1,-1):
        for u in range(len(states)):
            summ = 0
            for v in range(len(states)):
                summ += beta.iloc[v,i+1] * trans_params.loc[states[u],states[v]] * \
                        em_params.loc[x_vector[i],states[u]]
            beta.iloc[u,i] = summ
    
    return beta

def max_marginal_decode(x_vector,trans_params,em_params):
    
    alpha = get_forward_prob(x_vector,trans_params,em_params)
    beta = get_backward_prob(x_vector,trans_params,em_params)
    
    states = trans_params.index.tolist()
    states.remove('START')
    states.remove('O')
    states = ['O']+states
    
    prediction_indx = []
    
    for i in range(len(x_vector)):
        maxx = None
        argmax = None
        for u in range(len(states)):
            prob = alpha.iloc[u,i] * beta.iloc[u,i]
            if maxx is None:
                maxx = prob
                argmax = u
            elif prob > maxx:
                maxx = prob
                argmax = u
        prediction_indx.append(argmax)
        
    prediction = [states[indx] for indx in prediction_indx]
    return prediction

def decode_file(fin,fout,trans_params,em_params):
    word_bag = em_params.index.tolist()
    unlabelled_datas = get_unlabelled_data(fin)
    
    results = []
    for obs_vector in unlabelled_datas:
        copy = []
        for i in range(len(obs_vector)):
            if obs_vector[i] in word_bag:
                copy.append(obs_vector[i])
            else:
                copy.append('#UNK#')
        result = max_marginal_decode(copy,trans_params,em_params)
        assert len(result) == len(obs_vector)
        results.append(result)
    
    fout = file(fout,'w')
    for i in range(len(unlabelled_datas)):
        for j in range(len(unlabelled_datas[i])):
            x = unlabelled_datas[i][j]
            y = results[i][j]
            fout.write('{} {}\n'.format(x,y))
        fout.write('\n')
    fout.close
    print "max marginal decoding complete"



## Training and Decoding on EN data Results

In [4]:
print 'training ..'
data_dict = get_data('EN/train')
trans_params = get_transmission_params(data_dict)
em_params = get_modified_emission_params(data_dict,k=3)

print 'decoding ..'
decode_file('EN/dev.in','EN/dev.p4.out',trans_params,em_params)

training ..
decoding ..
max marginal decoding complete


```
>python3 evalResult.py EN/dev.out EN/dev.p4.out

#Entity in gold data: 226
#Entity in prediction: 180

#Correct Entity : 94
Entity  precision: 0.5222
Entity  recall: 0.4159
Entity  F: 0.4631

#Correct Sentiment : 57
Sentiment  precision: 0.3167
Sentiment  recall: 0.2522
Sentiment  F: 0.2808
```

## Training and Decoding on FR data Results

In [5]:
print 'training ..'
data_dict = get_data('FR/train')
trans_params = get_transmission_params(data_dict)
em_params = get_modified_emission_params(data_dict,k=3)

print 'decoding ..'
decode_file('FR/dev.in','FR/dev.p4.out',trans_params,em_params)

training ..
decoding ..
max marginal decoding complete


```
>python3 evalResult.py FR/dev.out FR/dev.p4.out

#Entity in gold data: 223
#Entity in prediction: 77

#Correct Entity : 26
Entity  precision: 0.3377
Entity  recall: 0.1166
Entity  F: 0.1733

#Correct Sentiment : 11
Sentiment  precision: 0.1429
Sentiment  recall: 0.0493
Sentiment  F: 0.0733
```